In [4]:
import pandas as pd
import plotly.express as px
from statsmodels.tsa.seasonal import seasonal_decompose
import logging
import datetime
import time

import dash
from dash import dcc, html
from dash.dependencies import Input, Output

import calendar
import pandas as pd
import plotly.express as px
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat, col
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
from ipywidgets import interact, widgets
from matplotlib.dates import MonthLocator, DateFormatter
import geopandas as gpd
from shapely.geometry import Point
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import geopandas as gpd
from mpl_toolkits.axes_grid1 import make_axes_locatable
#from fbprophet import Prophet
from sklearn.metrics import mean_squared_error
import numpy as np
import plotly.graph_objects as go
from pyspark.sql import functions as F
from pyspark.sql.functions import year, month
from sklearn.metrics.pairwise import euclidean_distances
from pyspark.sql.functions import year, month, sum as sum_
from scipy.spatial.distance import euclidean
from prophet import Prophet
from pyspark.sql.functions import col, countDistinct , desc , count , length
from pyspark.sql.functions import split, col, when
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, dayofmonth, last_day ,to_date, lit, isnull
from pyspark.sql.types import IntegerType
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error
from dateutil.relativedelta import relativedelta
from pyspark.sql.functions import date_format

import schedule
from pyspark.sql.functions import col, to_timestamp, date_format

import pyspark
import pandas as pd
import boto3
import sagemaker
import sagemaker.feature_store.feature_store as fs
import databricks.connect
from dateutil.relativedelta import relativedelta

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
from sklearn.metrics import mean_squared_error, root_mean_squared_error

import pandas as pd
import numpy as np
from prophet import Prophet
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
import plotly.graph_objects as go
from itertools import product
import random
import statsmodels.api as sm
import os

import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
from databricks.connect import DatabricksSession

# Get spark
spark = databricks.connect.DatabricksSession.builder.getOrCreate()

## Tabelas Usadas para Tabela Churn

#### dev.dw_redeok_model_aux.tb_s_status_transicao

In [5]:

churn_sdf_transicao = spark.table("dev.dw_redeok_model_aux.tb_s_status_transicao")

churn_sdf_transicao = churn_sdf_transicao.orderBy(col("data_transicao"))
churn_sdf_transicao.show(10000)
churn_sdf_transicao.count()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:4                                                                                    │
│                                                                                                  │
│   1 churn_sdf_transicao = spark.table("dev.dw_redeok_model_aux.tb_s_status_transicao")           │
│   2                                                                                              │
│   3 churn_sdf_transicao = churn_sdf_transicao.orderBy(col("data_transicao"))                     │
│ ❱ 4 churn_sdf_transicao.show(10000)                                                              │
│   5 churn_sdf_transicao.count()                                                                  │
│   6                                                                                              │
│                                                                                                  │
│ c:\Users\gusta\AppData\Local\pypoetry\Cache\virtualenvs\ltv-novo-aF9Dy8BA-py3.10\lib\site-packag │
│ es\pyspark\sql\connect\dataframe.py:1087 in show                                                 │
│                                                                                                  │
│   1084 │   observe.__doc__ = PySparkDataFrame.observe.__doc__                                    │
│   1085 │                                                                                         │
│   1086 │   def show(self, n: int = 20, truncate: Union[bool, int] = True, vertical: bool = Fals  │
│ ❱ 1087 │   │   print(self._show_string(n, truncate, vertical))                                   │
│   1088 │                                                                                         │
│   1089 │   show.__doc__ = PySparkDataFrame.show.__doc__                                          │
│   1090                                                                                           │
│                                                                                                  │
│ c:\Users\gusta\AppData\Local\pypoetry\Cache\virtualenvs\ltv-novo-aF9Dy8BA-py3.10\lib\site-packag │
│ es\pyspark\sql\connect\dataframe.py:831 in _show_string                                          │
│                                                                                                  │
│    828 │   │   │   │   vertical=vertical,                                                        │
│    829 │   │   │   ),                                                                            │
│    830 │   │   │   session=self._session,                                                        │
│ ❱  831 │   │   )._to_table()                                                                     │
│    832 │   │   return table[0][0].as_py()                                                        │
│    833 │                                                                                         │
│    834 │   def withColumns(self, colsMap: Dict[str, Column]) -> "DataFrame":                     │
│                                                                                                  │
│ c:\Users\gusta\AppData\Local\pypoetry\Cache\virtualenvs\ltv-novo-aF9Dy8BA-py3.10\lib\site-packag │
│ es\pyspark\sql\connect\dataframe.py:1772 in _to_table                                            │
│                                                                                                  │
│   1769 │                                                                                         │
│   1770 │   def _to_table(self) -> Tuple["pa.Table", Optional[StructType]]:                       │
│   1771 │   │   query = self._plan.to_proto(self._session.client)                                 │
│ ❱ 1772 │   │   table, schema = self._session.client.to_table(query, self._plan.observations)     │
│   1773 │   │   assert table is not None                    

In [3]:
# Contagem da relação entre 'previous' e 'motivo_cancelamento'
relacao = churn_sdf_transicao.groupBy("previous_status", "motivo_cancelamento").count()

# Exibir os resultados
relacao.orderBy(col("count").desc()).show()

+---------------+--------------------+------+
|previous_status| motivo_cancelamento| count|
+---------------+--------------------+------+
|              0|                NULL|709597|
|              1|INADIMPLENTE-AUTOMAT|554612|
|           NULL|                NULL|205424|
|              0|   FINANCEIRO/SERASA| 61367|
|              1|                NULL| 59945|
|              1|       INADIMPLENCIA| 50089|
|              1|BLOQ - TRAVA LMASTER| 31924|
|              1|       PRE CANCELADO| 29017|
|              1|MOTIVO NAO INFORMADO| 28135|
|              1|    FATURA EM ABERTO| 26037|
|             -3|                NULL| 23398|
|              0|  CONTRATO CANCELADO| 22483|
|              1|        INADIMPLENTE| 21815|
|             -1|                NULL| 21381|
|             -3|      EM-RECUPERACAO| 19072|
|              1| BLOQ. AUTO. 15 DIAS| 18233|
|              1| SOLICITACAO REVENDA| 17970|
|             -6|                NULL| 17569|
|           NULL|        INADIMPLE

- data de transição - Data na qual o contrato foi finalizado?
- todos contratos dessa tabela foram finalizados? ou aqueles que tem o motivo de canelamento Nulo não foram? Ou apenas não temos aquelas informações?
- porque um mesmo numero de previous_status represnetam diferentes motivos de cancelamento

#### dev.dw_redeok_model.tb_d_external_entity_association

In [4]:
churn_sdf_ass = spark.table("dev.dw_redeok_model.tb_d_external_entity_association")
churn_sdf_ass.show(1000)
churn_sdf_ass.count()

+--------------------+--------------------+--------------------+--------------------------+------------------+--------------------+-------------------------+-------------------------------+--------------------+
|                uuid|uuid_internal_entity|name_internal_entity|colum_name_internal_entity|id_external_entity|name_external_entity|name_external_application|colum_name_external_application|         insert_data|
+--------------------+--------------------+--------------------+--------------------------+------------------+--------------------+-------------------------+-------------------------------+--------------------+
|5db51586-f53c-498...|796c31ae-e4e2-446...|        tb_f_invoice|                      uuid|          10276856|     checkok.receber|                   signet|                         numero|2024-12-06 19:01:...|
|a17ddaa2-8216-409...|7aca4465-9d62-4da...|        tb_f_invoice|                      uuid|          10285369|     checkok.receber|                   signet

4860935

In [5]:
from pyspark.sql.functions import col

# Filtrando os registros que atendem às condições
churn_sdf_ass_filtrado = churn_sdf_ass.filter(
    (col("name_internal_entity") == "tb_d_contract") & 
    (col("name_external_application") == "signet")
)

# Exibir os resultados filtrados
churn_sdf_ass_filtrado.count()

60282

- 60 mil linhas são  enolvidas com tb_d_contract e signet que assim serão usadas em nosso codigo 


#### dev.dw_redeok_model.tb_d_contract

In [6]:
churn_sdf_contrato = spark.table("dev.dw_redeok_model.tb_d_contract")
#churn_sdf_contrato = churn_sdf_contrato.orderBy(col("start_date"))
churn_sdf_contrato.show(1000)
churn_sdf_contrato.count()

+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------+-----------------+----------+-------------+-------+-----------------+-------------------------+------------------+--------------------+------------------+--------------------+--------------------+-----------+---------------------+
|                uuid|       uuid_customer|        uuid_partner|         uuid_wallet| uuid_business_model|             status|         start_date|           end_date|status_reason|consumption_value|access_fee|financial_fee|due_day|payment_frequency|max_ticket_value_in_month|data_source_insert|          created_at|data_source_update|          updated_at|           uuid_user|entity_type|uuid_partner_contract|
+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+-------------------+-------------------+------------

61211

- Na tabelas contratos temos apenas alguns contratos (antes 2018 - fim depois de 2018 , depois de 2018 e fim depois de 2018)

#### dev.dw_redeok_model.tb_d_customer

In [7]:
churn_sdf_cliente = spark.table("dev.dw_redeok_model.tb_d_customer")
churn_sdf_cliente.show(1000)
churn_sdf_cliente.count()

+--------------------+--------------+-----+--------------------+------------------+--------------------+------------------+--------------------+
|                uuid|      cnpj_cpf|pj_pf|  company_civil_name|data_source_insert|          created_at|data_source_update|          updated_at|
+--------------------+--------------+-----+--------------------+------------------+--------------------+------------------+--------------------+
|a97f43f2-a613-4f8...|19248982000163|   PJ|O. R. COSTA IMPOR...|            signet|2024-02-21 13:50:...|                  |2024-02-21 13:50:...|
|3db38e3d-19c5-490...|81390619000232|   PJ|GLC MAQUINAS E FE...|            signet|2024-02-21 13:50:...|                  |2024-02-21 13:50:...|
|3c755551-c701-4de...|00735076000165|   PJ|SOLO FORTE EMPREE...|            signet|2024-02-21 13:50:...|                  |2024-02-21 13:50:...|
|3eceea78-d712-4e8...|03336700000140|   PJ|IMOBILIARIA IVAN ...|            signet|2024-02-21 13:50:...|                  |2024-02

57147

## Tabela Churn

In [6]:
churn_sdf_1 = spark.table("dev.dw_redeok_model.tb_f_churn")

# Ordenando pela coluna churn_date da mais antiga para a mais nova
churn_sdf_1 = churn_sdf_1.orderBy(col("churn_date").asc())

# Exibindo os dados ordenados
churn_sdf_1.show()
churn_sdf_1.count()

+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+------------------+--------------------+------------------+--------------------+
|                uuid|       uuid_contract|       uuid_customer|         churn_date|          motivation|          churn_type|data_source_insert|          created_at|data_source_update|          updated_at|
+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+------------------+--------------------+------------------+--------------------+
|2c66166c-adc3-4e1...|62e395ab-f114-45b...|f3ce93b1-8097-491...|2021-01-03 11:30:20|      OUTROS MOTIVOS|   90 DIAS BLOQUEADO|            signet|2024-04-26 20:36:...|                  |2024-04-26 20:36:...|
|647fa0fe-52f0-414...|988898d4-ec69-46b...|59040f2a-cd12-48a...|2021-01-03 20:13:20|                NULL|90 DIAS PRÉ CANCE...|            signet|2024-04-26 20:36:...|      

24296

## Tabela Churn por Partes

## Parte 1

In [2]:
query_1=  """ WITH transicao_global AS (
    SELECT
        cnt.uuid AS uuid_contract,
        str.matriz AS matriz,
        str.data_transicao AS data_transicao,
        str.previous_status AS previous_status,
        str.next_status AS next_status,
        str.motivo_cancelamento AS motivo_cancelamento
    FROM
        dev.dw_redeok_model_aux.tb_s_status_transicao str
    LEFT JOIN dev.dw_redeok_model.tb_d_external_entity_association ext 
        ON str.matriz = ext.id_external_entity
        AND name_internal_entity = 'tb_d_contract'
        AND ext.name_external_application = 'signet'
    LEFT JOIN dev.dw_redeok_model.tb_d_contract cnt
        ON ext.uuid_internal_entity = cnt.uuid
)
SELECT * FROM transicao_global;

"""

# Executando a consulta e obtendo os dados em um DataFrame Spark
query_1= spark.sql(query_1)
query_1 = query_1.orderBy(col("data_transicao"))
query_1.show()

contagem_total =query_1.count()

# Contar os registros onde uuid_contract é nulo
contagem_nulos = query_1.filter(col("uuid_contract").isNull()).count()

# Exibir o resultado
print(f"Quantidade de registros Total: {contagem_total}")
print(f"Quantidade de registros com uuid_contract nulo: {contagem_nulos}")


+-------------+------+--------------------+---------------+-----------+-------------------+
|uuid_contract|matriz|      data_transicao|previous_status|next_status|motivo_cancelamento|
+-------------+------+--------------------+---------------+-----------+-------------------+
|         NULL|134220|2006-01-02 09:11:...|           NULL|          1|               NULL|
|         NULL|157709|2006-01-02 09:17:...|           NULL|          1|               NULL|
|         NULL| 20452|2006-01-02 09:22:...|           NULL|          1|               NULL|
|         NULL|123951|2006-01-02 09:22:...|           NULL|         -1| CONTRATO CANCELADO|
|         NULL|145233|2006-01-02 09:22:...|           NULL|          1|               NULL|
|         NULL|121504|2006-01-02 09:33:...|           NULL|         -1| CONTRATO CANCELADO|
|         NULL|111140|2006-01-02 09:38:...|           NULL|         -1| CONTRATO CANCELADO|
|         NULL|121997|2006-01-02 09:46:...|           NULL|          1|         

**Dúvidas:**
**Alta quantidade de valores nulos em contrato_id após o JOIN**

- Ao realizar o JOIN com a tabela de contratos, notei uma grande quantidade de valores nulos na coluna contrato_id.
Isso indica que, de um total de 1000000 contratos, cerca da metade não pôde ser vinculada a nenhum contrato existente.
Identificação dos contratos não vinculados

- É possível identificar quais são esses contratos?
- Há alguma forma de entender o motivo pelo qual esses contratos não estão sendo vinculados?


**Existe alguma tabela de contratos que contemple os seguintes casos?**
- Contratos iniciados antes de 2018 e encerrados depois de 2018.
- Contratos iniciados depois de 2018.
- Contratos iniciados antes de 2018 e encerrados antes de 2018.
- Necessidade de uma base completa de contratos

- Para calcular corretamente o churn e o tempo de permanência, precisamos de uma tabela completa contendo todos os contratos de todos os anos.
Caso contrário, a análise pode ficar enviesada, afetando a interpretação dos resultados.

In [3]:
from pyspark.sql.functions import col, count

# Contar a quantidade de ocorrências de cada valor na coluna "matriz"
duplicados = query_1.groupBy("matriz").count().filter(col("count") > 1)

# Exibir os valores duplicados
x = duplicados.count()

print(f"Quantidade de matriz com mais de um contrato: {x}")


Quantidade de matriz com mais de um contrato: 201956


- Temos 201.956 contratos associados a mais de uma matriz nesta primeira etapa da construção da tabela de churn.

## Parte 2

In [12]:
query_2=  """WITH transicao_global AS (
    SELECT 
        cnt.uuid AS uuid_contract,
        str.matriz AS matriz,
        str.data_transicao AS data_transicao,
        str.previous_status AS previous_status,
        str.next_status AS next_status,
        str.motivo_cancelamento AS motivo_cancelamento
    FROM dev.dw_redeok_model_aux.tb_s_status_transicao str
    LEFT JOIN dev.dw_redeok_model.tb_d_external_entity_association ext 
        ON str.matriz = ext.id_external_entity
        AND ext.name_internal_entity = 'tb_d_contract'
        AND ext.name_external_application = 'signet'
    LEFT JOIN dev.dw_redeok_model.tb_d_contract cnt
        ON ext.uuid_internal_entity = cnt.uuid
),

bloqueio AS (
    SELECT 
        uuid_contract,
        matriz,
        data_transicao,
        previous_status,
        next_status,
        motivo_cancelamento,
        ROW_NUMBER() OVER (PARTITION BY matriz ORDER BY data_transicao DESC) AS row_num
    FROM transicao_global
)

SELECT 
    uuid_contract,
    matriz,
    data_transicao,
    previous_status,
    next_status,
    motivo_cancelamento
FROM bloqueio
WHERE row_num = 1

"""

# Supondo que o resultado da query seja armazenado em um DataFrame do Spark
query_2 = spark.sql(query_2)

query_2.show()

contagem_total =query_2.count()

# Contar os registros onde uuid_contract é nulo
contagem_nulos = query_2.filter(col("uuid_contract").isNull()).count()

# Exibir o resultado
print(f"Quantidade de registros Total: {contagem_total}")
print(f"Quantidade de registros com uuid_contract nulo: {contagem_nulos}")

+-------------+------+--------------------+---------------+-----------+-------------------+
|uuid_contract|matriz|      data_transicao|previous_status|next_status|motivo_cancelamento|
+-------------+------+--------------------+---------------+-----------+-------------------+
|         NULL|100008|2006-02-02 18:18:...|           NULL|         -1| CONTRATO CANCELADO|
|         NULL|100083|2011-10-20 11:24:...|             -3|         -1|      PRE CANCELADO|
|         NULL|100090|2006-02-02 18:18:...|           NULL|         -1| CONTRATO CANCELADO|
|         NULL|100103|2006-07-28 15:51:...|              0|         -1| CONTRATO CANCELADO|
|         NULL|100111|2006-02-02 18:18:...|           NULL|         -1| CONTRATO CANCELADO|
|         NULL|100305|2008-12-18 08:38:...|              1|         -1|      PRE CANCELADO|
|         NULL|100320|2006-02-02 18:18:...|           NULL|         -1| CONTRATO CANCELADO|
|         NULL|100332|2015-01-28 11:36:...|              0|         -1| CONTRATO

**Redução significativa no número de linhas**

- Houve uma redução considerável na quantidade de linhas.
- Inicialmente, havia cerca de 2 milhões de registros, e agora restam 250 mil.Isso ocorreu porque estamos considerando apenas os contratos com a data de transição mais recente vinculada.
- 

**Impacto na análise de negócios**

- Ao remover os contratos com as datas de transição (cancelamentos) mais recentes, os contratos eliminados correspondem a quê?
- São contratos da mesma matriz, mas com datas de churn menos recentes?

**Valores nulos**
- A quantidade de valores nulos no contrato ainda é muito grande na faixa de 75 % do dataset

In [ ]:
from pyspark.sql.functions import col, count

# Contar a quantidade de ocorrências de cada valor na coluna "matriz"
duplicados = query_2.groupBy("matriz").count().filter(col("count") > 1)

# Exibir os valores duplicados
x = duplicados.count()

print(f"Quantidade de matriz com mais de um contrato: {x}")

+------+-----+
|matriz|count|
+------+-----+
+------+-----+



**Por que isso ainda ocorre?**

- Se todos os contratos da mesma matriz foram removidos e ficaram apenas os mais recentes, o que pode estar causando a persistência desses contratos sem contract_uuid?
- E se mantivéssemos todos os contratos como antes? Isso permitiria visualizar todas as entradas e saídas de todas as contas e contratos que já tivemos?

### Parte 3

In [2]:
# Criando a Query Corrigida
query_3 = """
WITH transicao_global AS (
      SELECT 
              cnt.uuid AS uuid_contract,
              str.matriz AS matriz,
              str.data_transicao AS data_transicao,
              str.previous_status AS previous_status,
              str.next_status AS next_status,
              str.motivo_cancelamento AS motivo_cancelamento
      FROM dev.dw_redeok_model_aux.tb_s_status_transicao str
      LEFT JOIN dev.dw_redeok_model.tb_d_external_entity_association ext 
            ON str.matriz = ext.id_external_entity
            AND ext.name_internal_entity = 'tb_d_contract'
            AND ext.name_external_application = 'signet'
      LEFT JOIN dev.dw_redeok_model.tb_d_contract cnt
            ON ext.uuid_internal_entity = cnt.uuid
),

bloqueio AS (
  SELECT 
         uuid_contract,
         matriz,
         data_transicao,
         previous_status,
         next_status,
         motivo_cancelamento,
         row_number() OVER (PARTITION BY matriz ORDER BY data_transicao DESC) AS row_num
  FROM transicao_global
),

cancelamento_noventa_dias_bloqueio AS (
  SELECT 
         uuid_contract,
         matriz,
         data_transicao,
         date_add(data_transicao, 90) AS data_transicao_90d,  -- Ajuste para Spark
         previous_status,
         next_status,
         motivo_cancelamento
  FROM bloqueio
  WHERE row_num = 1 AND next_status IN (-7, -5, 0)
)

SELECT * FROM cancelamento_noventa_dias_bloqueio
"""

# Executando no Spark
query_3 = spark.sql(query_3)

contagem_total =query_3.count()

# Contar os registros onde uuid_contract é nulo
contagem_nulos = query_3.filter(col("uuid_contract").isNull()).count()

# Exibir o resultado
print(f"Quantidade de registros Total: {contagem_total}")
print(f"Quantidade de registros com uuid_contract nulo: {contagem_nulos}")

Quantidade de registros Total: 31415
Quantidade de registros com uuid_contract nulo: 14439


In [3]:
query_3.show()

+--------------------+------+--------------------+------------------+---------------+-----------+--------------------+
|       uuid_contract|matriz|      data_transicao|data_transicao_90d|previous_status|next_status| motivo_cancelamento|
+--------------------+------+--------------------+------------------+---------------+-----------+--------------------+
|dcb352fd-5d11-481...|100560|2018-04-24 15:57:...|        2018-07-23|              1|          0|INADIMPLENTE-AUTOMAT|
|                NULL|100726|2006-12-29 12:09:...|        2007-03-29|           NULL|          0|                NULL|
|                NULL|104056|2008-11-24 09:42:...|        2009-02-22|              1|          0|INADIMPLENTE-AUTOMAT|
|c68d0b1a-ab0a-40e...|104758|2025-01-27 08:30:...|        2025-04-27|              1|          0| BLOQ. AUTO. 15 DIAS|
|a33f598c-af84-46f...|104856|2018-01-23 15:30:...|        2018-04-23|              1|          0|INADIMPLENTE-AUTOMAT|
|8eb37871-95c8-4f8...|106185|2020-07-24 11:15:..

-  Agora, o número de registros reduziu de 250 mil para 31 mil, e a quantidade de contratos nulos representa quase metade do total. Isso ocorre porque, neste filtro, mantivemos apenas os contratos mais recentes e aqueles com next_status igual a -7, -5 ou 0.

## Parte 4

In [28]:
query_4 = """
WITH transicao_global AS (
      SELECT 
              cnt.uuid AS uuid_contract,
              str.matriz AS matriz,
              str.data_transicao AS data_transicao,
              str.previous_status AS previous_status,
              str.next_status AS next_status,
              str.motivo_cancelamento AS motivo_cancelamento
      FROM dev.dw_redeok_model_aux.tb_s_status_transicao str
      LEFT JOIN dev.dw_redeok_model.tb_d_external_entity_association ext 
            ON str.matriz = ext.id_external_entity
            AND ext.name_internal_entity = 'tb_d_contract'
            AND ext.name_external_application = 'signet'
      LEFT JOIN dev.dw_redeok_model.tb_d_contract cnt
            ON ext.uuid_internal_entity = cnt.uuid
),

bloqueio AS (
  SELECT 
         uuid_contract,
         matriz,
         data_transicao,
         previous_status,
         next_status,
         motivo_cancelamento,
         row_number() OVER (PARTITION BY matriz ORDER BY data_transicao DESC) AS row_num
  FROM transicao_global
),

cancelamento_noventa_dias_bloqueio AS (
  SELECT 
         uuid_contract,
         matriz,
         data_transicao,
         date_add(data_transicao, 90) AS data_transicao_90d,  -- Ajuste para Spark
         previous_status,
         next_status,
         motivo_cancelamento
  FROM bloqueio
  WHERE row_num = 1 AND next_status IN (-7, -5, 0)
),

cancelamento_normal_bloqueio_90d AS (
  SELECT
         uuid_contract,
         matriz,
         data_transicao AS churn_date,
         previous_status,
         next_status,
         motivo_cancelamento,
         'CANCELAMENTO NORMAL' AS tipo_saida
  FROM transicao_global
  WHERE next_status = -1
)

SELECT * FROM cancelamento_normal_bloqueio_90d
"""

# Executando no Spark
query_4 = spark.sql(query_4)

contagem_total =query_4.count()

# Contar os registros onde uuid_contract é nulo
contagem_nulos = query_4.filter(col("uuid_contract").isNull()).count()

# Exibir o resultado
print(f"Quantidade de registros Total: {contagem_total}")
print(f"Quantidade de registros com uuid_contract nulo: {contagem_nulos}")


Quantidade de registros Total: 195043
Quantidade de registros com uuid_contract nulo: 158216


- Nesta etapa, não filtramos apenas os contratos mais recentes, pois utilizamos a CTE transição global, que considera todos os registros. No entanto, aplicamos um filtro para manter apenas aqueles com next_status = -1.

## Parte 5 (Final)

In [35]:
query_global=  """


with

transicao_global as
(
      select
              cnt.uuid                                                                        as uuid_contract
            , str.matriz                                                                      as matriz
            , str.data_transicao                                                              as data_transicao
            , str.previous_status                                                             as previous_status
            , str.next_status                                                                 as next_status
            , str.motivo_cancelamento                                                         as motivo_cancelamento
      from
            dev.dw_redeok_model_aux.tb_s_status_transicao str
            --
            left join dev.dw_redeok_model.tb_d_external_entity_association ext 
                  on str.matriz = ext.id_external_entity
                  and name_internal_entity = 'tb_d_contract'
                  and ext.name_external_application = 'signet'
            --
            left join dev.dw_redeok_model.tb_d_contract cnt
                  on ext.uuid_internal_entity = cnt.uuid
),
bloqueio as (
  select
           uuid_contract
        ,  matriz
        , data_transicao
        , previous_status
        , next_status
        , motivo_cancelamento
        , row_number() over (partition by matriz order by data_transicao desc)                as row_num
  from
        transicao_global
        qualify row_number() over (partition by matriz order by data_transicao desc) = 1
),

cancelamento_noventa_dias_bloqueio as 
(
  select 
          uuid_contract                                                                       as uuid_contract
        , matriz                                                                              as matriz
        , data_transicao                                                                      as data_transicao
        , dateadd(DAY, 90, data_transicao)                                                    as data_transicao_90d
        , previous_status                                                                     as previous_status
        , next_status                                                                         as next_status
        , motivo_cancelamento                                                                 as motivo_cancelamento
  from
        bloqueio
  where
        next_status in (-7, -5, 0)
),
cancelamento_normal_bloqueio_90d as
(
  select
          uuid_contract                                                                       as uuid_contract
        , matriz                                                                              as matriz
        , data_transicao                                                                      as churn_date
        , previous_status                                                                     as previous_status
        , next_status                                                                         as next_status
        , motivo_cancelamento                                                                 as motivo_cancelamento
        , 'CANCELAMENTO NORMAL'                                                               as tipo_saida
  from
      transicao_global
  where
        next_status = -1

union

select
        uuid_contract                                                                          as uuid_contract
      , matriz                                                                                 as matriz
      , data_transicao_90d                                                                     as churn_date
      , previous_status                                                                        as previous_status
      , next_status                                                                            as next_status
      , motivo_cancelamento                                                                    as motivo_cancelamento
      , case
            --when date_add(data_transicao_90d, 1) > current_date()         then 'PROJEÇÃO BLOQUEIO'
            when next_status = 0                                          then '90 DIAS BLOQUEADO'
            when next_status = -5                                         then '90 DIAS PRÉ CANCELADO'
            when next_status = -7                                         then '90 DIAS PRÉ CANCELADO JURÍDICO'
            else null
      end                                                                                      as tipo_saida
from
      cancelamento_noventa_dias_bloqueio
where
      date(data_transicao_90d) < current_date()

)
select
        cast(uuid() as string)                                                                  as uuid
      , cnt.uuid                                                                                as uuid_contract
      , cnt.uuid_customer                                                                       as uuid_customer
      , cast(can.churn_date as timestamp)                                                       as churn_date
      , trim(can.motivo_cancelamento)                                                           as motivation
      , can.tipo_saida                                                                          as churn_type
      , 'signet'                                                                                as data_source_insert
      , now()                                                                                   as created_at
      , ''                                                                                      as data_source_update
      , now()                                                                                   as updated_at
from
      cancelamento_normal_bloqueio_90d can
      --
      inner join dev.dw_redeok_model.tb_d_contract cnt
             on can.uuid_contract = cnt.uuid
      --
      inner join dev.dw_redeok_model.tb_d_customer as cus
            on cnt.uuid_customer = cus.uuid
where
      date(can.churn_date) >= date('2024-04-16') -- ultima data utilizada com base exportada do bigquery 
      and upper(trim(can.motivo_cancelamento)) not in ( 
                                                            'TROCA DE CNPJ',
                                                            'TROCA CNPJ',
                                                            'TROCA DE RECEITA',
                                                            'TROCA DE CARTEIRA',
                                                            'TROCA RECEITA',
                                                            'TROCA DE MODALIDADE')
     
      order by can.uuid_contract desc


      """


# Executando a consulta e obtendo os dados em um DataFrame Spark
query_global= spark.sql(query_global)
query_global = query_global.orderBy(col("churn_date"))
query_global.show()

contagem_total =query_global.count()

# Contar os registros onde uuid_contract é nulo
contagem_nulos = query_global.filter(col("uuid_contract").isNull()).count()

# Exibir o resultado
print(f"Quantidade de registros Total: {contagem_total}")
print(f"Quantidade de registros com uuid_contract nulo: {contagem_nulos}")

+--------------------+--------------------+--------------------+-------------------+-------------------+-------------------+------------------+--------------------+------------------+--------------------+
|                uuid|       uuid_contract|       uuid_customer|         churn_date|         motivation|         churn_type|data_source_insert|          created_at|data_source_update|          updated_at|
+--------------------+--------------------+--------------------+-------------------+-------------------+-------------------+------------------+--------------------+------------------+--------------------+
|ec246990-5cee-489...|778962b3-fcae-412...|a0780640-c79e-411...|2024-04-16 10:19:43|       RECLAME AQUI|CANCELAMENTO NORMAL|            signet|2025-02-07 21:41:...|                  |2025-02-07 21:41:...|
|00bde1bc-e0c5-42c...|87cf322c-73fd-4d1...|3ed380bb-7839-42f...|2024-04-16 10:31:05|ALT. POLIT DE VENDA|CANCELAMENTO NORMAL|            signet|2025-02-07 21:41:...|                

In [37]:
query_global.groupBy("churn_type").count().orderBy(col("count").desc()).show()

+--------------------+-----+
|          churn_type|count|
+--------------------+-----+
| CANCELAMENTO NORMAL| 4320|
|   90 DIAS BLOQUEADO|  400|
|90 DIAS PRÉ CANCE...|  114|
|90 DIAS PRÉ CANCE...|    1|
+--------------------+-----+



In [38]:
query_global.groupBy("motivation").count().orderBy(col("count").desc()).show()

+--------------------+-----+
|          motivation|count|
+--------------------+-----+
|   BLOQUEIO_REGRA_90| 1748|
|   ACAO DSPAC/FATMIN|  742|
|  Cadastro duplicado|  209|
|    ALT POLIT VENDAS|  194|
| ALT. POLIT DE VENDA|  154|
|            INADIMPL|  140|
| NAO ESTA UTILIZANDO|  122|
|MOTIVO NAO INFORMADO|  114|
|          NEGATIVADO|  104|
| BLOQ. AUTO. 15 DIAS|  100|
|              PRECOS|   91|
|            ENC ATIV|   91|
|      SOLIC PARCEIRO|   82|
|   BLOQ TRAVA MANUAL|   78|
|       CL NUNCA USOU|   74|
|       ENCER DE ATIV|   72|
| SOLICITACAO REVENDA|   72|
|         CONC SERASA|   69|
|          INSAT SERV|   68|
|      CNPJ IRREGULAR|   38|
+--------------------+-----+
only showing top 20 rows



In [39]:
query_global_1=  """


WITH transicao_global AS (
    SELECT 
        cnt.uuid AS uuid_contract,
        str.matriz AS matriz,
        str.data_transicao AS data_transicao,
        str.previous_status AS previous_status,
        str.next_status AS next_status,
        str.motivo_cancelamento AS motivo_cancelamento
    FROM 
        dev.dw_redeok_model_aux.tb_s_status_transicao str
    LEFT JOIN dev.dw_redeok_model.tb_d_external_entity_association ext 
        ON str.matriz = ext.id_external_entity
        AND name_internal_entity = 'tb_d_contract'
        AND ext.name_external_application = 'signet'
    LEFT JOIN dev.dw_redeok_model.tb_d_contract cnt
        ON ext.uuid_internal_entity = cnt.uuid
),

bloqueio AS (
    SELECT 
        uuid_contract,
        matriz,
        data_transicao,
        previous_status,
        next_status,
        motivo_cancelamento,
        ROW_NUMBER() OVER (PARTITION BY matriz ORDER BY data_transicao DESC) AS row_num
    FROM 
        transicao_global
    QUALIFY ROW_NUMBER() OVER (PARTITION BY matriz ORDER BY data_transicao DESC) = 1
),

cancelamento_noventa_dias_bloqueio AS (
    SELECT 
        uuid_contract,
        matriz,
        data_transicao,
        DATEADD(DAY, 90, data_transicao) AS data_transicao_90d,
        previous_status,
        next_status,
        motivo_cancelamento
    FROM 
        bloqueio
    WHERE 
        next_status IN (-7, -5, 0)
),

cancelamento_normal_bloqueio_90d AS (
    SELECT 
        uuid_contract,
        matriz,
        data_transicao AS churn_date,
        previous_status,
        next_status,
        motivo_cancelamento,
        'CANCELAMENTO NORMAL' AS tipo_saida
    FROM 
        transicao_global
    WHERE 
        next_status = -1

    UNION 

    SELECT 
        uuid_contract,
        matriz,
        data_transicao_90d AS churn_date,
        previous_status,
        next_status,
        motivo_cancelamento,
        CASE 
            WHEN next_status = 0 THEN '90 DIAS BLOQUEADO'
            WHEN next_status = -5 THEN '90 DIAS PRÉ CANCELADO'
            WHEN next_status = -7 THEN '90 DIAS PRÉ CANCELADO JURÍDICO'
            ELSE NULL
        END AS tipo_saida
    FROM 
        cancelamento_noventa_dias_bloqueio
    WHERE 
        DATE(data_transicao_90d) < CURRENT_DATE()
)

SELECT 
    CAST(UUID() AS STRING)                   AS uuid,
    cnt.uuid                                 AS uuid_contract,
    cnt.uuid_customer                        AS uuid_customer,
    CAST(can.churn_date AS TIMESTAMP)        AS churn_date,
    TRIM(can.motivo_cancelamento)            AS motivation,
    can.tipo_saida                           AS churn_type,
    'signet' AS data_source_insert,
    NOW() AS created_at,
    '' AS data_source_update,
    NOW() AS updated_at
FROM 
    cancelamento_normal_bloqueio_90d can
INNER JOIN dev.dw_redeok_model.tb_d_contract cnt
    ON can.uuid_contract = cnt.uuid
INNER JOIN dev.dw_redeok_model.tb_d_customer cus
    ON cnt.uuid_customer = cus.uuid
ORDER BY 
    can.uuid_contract DESC;

      """


# Executando a consulta e obtendo os dados em um DataFrame Spark
query_global_1= spark.sql(query_global_1)
query_global_1 = query_global.orderBy(col("churn_date"))
query_global_1.show()

contagem_total =query_global_1.count()

# Contar os registros onde uuid_contract é nulo
contagem_nulos = query_global_1.filter(col("uuid_contract").isNull()).count()

# Exibir o resultado
print(f"Quantidade de registros Total: {contagem_total}")
print(f"Quantidade de registros com uuid_contract nulo: {contagem_nulos}")

+--------------------+--------------------+--------------------+-------------------+-------------------+-------------------+------------------+--------------------+------------------+--------------------+
|                uuid|       uuid_contract|       uuid_customer|         churn_date|         motivation|         churn_type|data_source_insert|          created_at|data_source_update|          updated_at|
+--------------------+--------------------+--------------------+-------------------+-------------------+-------------------+------------------+--------------------+------------------+--------------------+
|182d1b63-a50b-4d4...|778962b3-fcae-412...|a0780640-c79e-411...|2024-04-16 10:19:43|       RECLAME AQUI|CANCELAMENTO NORMAL|            signet|2025-02-07 21:56:...|                  |2025-02-07 21:56:...|
|585d7d8c-dce8-4fc...|87cf322c-73fd-4d1...|3ed380bb-7839-42f...|2024-04-16 10:31:05|ALT. POLIT DE VENDA|CANCELAMENTO NORMAL|            signet|2025-02-07 21:56:...|                

In [40]:
query_global_1.groupBy("churn_type").count().orderBy(col("count").desc()).show()

+--------------------+-----+
|          churn_type|count|
+--------------------+-----+
| CANCELAMENTO NORMAL| 4320|
|   90 DIAS BLOQUEADO|  400|
|90 DIAS PRÉ CANCE...|  114|
|90 DIAS PRÉ CANCE...|    1|
+--------------------+-----+



In [41]:
query_global_1.groupBy("motivation").count().orderBy(col("count").desc()).show()

+--------------------+-----+
|          motivation|count|
+--------------------+-----+
|   BLOQUEIO_REGRA_90| 1748|
|   ACAO DSPAC/FATMIN|  742|
|  Cadastro duplicado|  209|
|    ALT POLIT VENDAS|  194|
| ALT. POLIT DE VENDA|  154|
|            INADIMPL|  140|
| NAO ESTA UTILIZANDO|  122|
|MOTIVO NAO INFORMADO|  114|
|          NEGATIVADO|  104|
| BLOQ. AUTO. 15 DIAS|  100|
|              PRECOS|   91|
|            ENC ATIV|   91|
|      SOLIC PARCEIRO|   82|
|   BLOQ TRAVA MANUAL|   78|
|       CL NUNCA USOU|   74|
|       ENCER DE ATIV|   72|
| SOLICITACAO REVENDA|   72|
|         CONC SERASA|   69|
|          INSAT SERV|   68|
|      CNPJ IRREGULAR|   38|
+--------------------+-----+
only showing top 20 rows



**Parte 1**

- tabelas envolvidas :  dev.dw_redeok_model_aux.tb_s_status_transicao , dev.dw_redeok_model.tb_d_external_entity_association , dev.dw_redeok_model.tb_d_contract
- A tabela tb_s_status_transicao é selecionada : UUID do contrato , identificador da matriz da transição , Data da mudança de status, Status anterior do contrato , Status seguinte do contrato , Motivo do cancelamento (se houver)
- As tabelas são unidas utilizando LEFT JOIN para garantir que todas as linhas da tabela tb_s_status_transicao sejam mantidas, mesmo se não houver correspondência nas tabelas associadas tb_d_external_entity_association.
- Aqui, matriz da transição de status (str.matriz) é comparado com id_external_entity da tabela de associação (ext).
- A condição name_internal_entity = 'tb_d_contract' filtra apenas os registros relacionados a contratos.
- name_external_application = 'signet' garante que apenas transições vinculadas ao sistema "signet" sejam consideradas.
- por fim faz um left join uuid_internal_entity na tabela de associação (ext) é comparado com uuid da tabela de contratos (cnt).garante que cada transição de status seja vinculada ao contrato correto.

**Parte 2**
- Agora seleciona os campus que faz parte da data de transição do status do contrato da primeira tabela gerada
- A função ROW_NUMBER() numera as linhas dentro de cada matriz (agrupamento por matriz).
- O critério de ordenação é data_transicao DESC, ou seja, da data mais recente para a mais antiga.
- O resultado é que a transição mais recente recebe o número 1, a segunda mais recente recebe 2, e assim por diante.
- O QUALIFY é um filtro que mantém apenas as linhas onde row_number() = 1, ou seja, apenas a transição mais recente para cada matriz
- em Resumo Selecionamos apenas a data de transição mais recente para cada matriz utilizando QUALIFY.

**Parte 3**
- Esses campos são mantidos exatamente como estão na CTE bloqueio : uuid_contract , matriz , data_transicao , previous_status , next_status e  motivo_cancelamento
- Adciona uma coluna onde calcula a data de transição + 90 dias , O resultado é armazenado na nova coluna data_transicao_90d.
- Depois filtramos apenas as transições cujo next_status são -7 , -5 e 0 Isso significa que não estamos considerando todas as transições de status, mas apenas essas específicas.
- Em suma Selecionamos a transição mais recente de cada matriz (vinda da CTE bloqueio) , Calculamos a data de transição acrescida de 90 dias (data_transicao_90d) e  Filtramos apenas os registros com next_status igual a -7, -5 ou 0. (Não entendi o porque)

**Parte 4**
- Esta CTE (cancelamento_normal_bloqueio_90d) seleciona registros da tabela transicao_global que possuem next_status = -1, identificando contratos que passaram por um cancelamento normal.
- A data_transicao foi renomeada para churn_date para indicar que representa a data do cancelamento.Porem o restante das variaveis foi mantida igual
- A consulta adiciona manualmente uma nova coluna chamada tipo_saida, onde todas as linhas terão o valor 'CANCELAMENTO NORMAL'. Isso indica que esse tipo de cancelamento segue um processo padrão dentro do sistema
- Apenas os contratos cuja resultou em next_status = -1 são mantidos.Isso significa que esses contratos foram cancelados normalmente.

**Parte 5 - Final (Union)**

**União das transições de cancelamento**
- Essa query junta todas as CTEs anteriores (cancelamento_noventa_dias_bloqueio e cancelamento_normal_bloqueio_90d), aplica regras de categorização dos cancelamentos e filtra os contratos que não devem ser considerados, além de evitar duplicidade na tabela de churn.
- Seleciona dados da CTE cancelamento_noventa_dias_bloqueio (cancelamentos após 90 dias de bloqueio).
- Renomeia data_transicao_90d para churn_date (indicando a data do churn).
- Cria uma categorização (tipo_saida) baseada no next_status : 0 → 90 DIAS BLOQUEADO , -5 → 90 DIAS PRÉ CANCELADO , -7 → 90 DIAS PRÉ CANCELADO JURÍDICO , (data_transicao_90d, 1) > current_date() →  'PROJEÇÃO BLOQUEIO'
- Filtra contratos onde data_transicao_90d seja menor que a data atual, garantindo que apenas eventos passados sejam considerados.


**Seleção dos dados finais**
- Cria um UUID único (uuid()) para cada linha.
- Traz os dados da CTE cancelamento_normal_bloqueio_90d (cancelamentos normais).
- Adiciona metadados:
churn_date formatado corretamente.
motivation: Motivo do cancelamento (sem espaços extras).
churn_type: Tipo do churn, categorizado previamente.
data_source_insert: Identifica que os dados vêm da plataforma "signet".
created_at e updated_at: Timestamps da inserção e atualização.


**Junção com tabelas de contratos e clientes**
- Junta os dados do contrato (tb_d_contract) e os dados do cliente (tb_d_customer).
- Objetivo: Garantir que estamos vinculando o churn aos contratos e clientes corretos.


**Filtros de data e remoção de churn duplicado**
- Filtra churns a partir de 16 de abril de 2024 (DATE(can.churn_date) >= DATE('2024-04-16')).
- Remove registros com motivos de cancelamento indesejados, como:Troca de CNPJ , Troca de Receita , Troca de Carteira e Troca de Modalidade


**Verificação para evitar duplicidade na tabela final**
- Evita inserir churns duplicados.
- Verifica se já existe um churn para o mesmo contrato e data na tabela tb_f_churn. Se já existir, o contrato não será incluído novamente.
- Ordena os contratos em ordem decrescente de UUID para exibir os mais recentes primeiro.



**Resumo**
- Unimos os cancelamentos de 90 dias e cancelamentos normais.
- Criamos categorias para os cancelamentos (tipo_saida).
- Vinculamos os contratos e clientes às transições de churn.
- Filtramos por data e removemos cancelamentos irrelevantes.
- Garantimos que não inserimos churns duplicados.
- Ordenamos os dados de forma decrescente pelo UUID do contrato.

## Tabela Churn

In [2]:
churn_sdf_1 = spark.table("dev.dw_redeok_model.tb_f_churn")

# Ordenando pela coluna churn_date da mais antiga para a mais nova
churn_sdf_1 = churn_sdf_1.orderBy(col("churn_date").asc())

# Exibindo os dados ordenados
churn_sdf_1.show()
churn_sdf_1.count()

+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+------------------+--------------------+------------------+--------------------+
|                uuid|       uuid_contract|       uuid_customer|         churn_date|          motivation|          churn_type|data_source_insert|          created_at|data_source_update|          updated_at|
+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+------------------+--------------------+------------------+--------------------+
|2c66166c-adc3-4e1...|62e395ab-f114-45b...|f3ce93b1-8097-491...|2021-01-03 11:30:20|      OUTROS MOTIVOS|   90 DIAS BLOQUEADO|            signet|2024-04-26 20:36:...|                  |2024-04-26 20:36:...|
|647fa0fe-52f0-414...|988898d4-ec69-46b...|59040f2a-cd12-48a...|2021-01-03 20:13:20|                NULL|90 DIAS PRÉ CANCE...|            signet|2024-04-26 20:36:...|      

24296

In [3]:
from pyspark.sql.functions import col



# Filtrando os churn_date maiores que '2024-04-16'
churn_filtered = churn_sdf_1.filter(col("churn_date") >= "2024-04-16")

# Contando os registros que atendem à condição
count_churn = churn_filtered.count()

# Exibindo o resultado
print(f"Número de churns após 2024-04-16: {count_churn}")


Número de churns após 2024-04-16: 6259


In [33]:
from pyspark.sql.functions import min, max

# Calculando a data mais antiga e mais recente
churn_sdf_1.select(
    min("churn_date").alias("data_mais_antiga"),
    max("churn_date").alias("data_mais_recente")
).show()


+-------------------+-------------------+
|   data_mais_antiga|  data_mais_recente|
+-------------------+-------------------+
|2021-01-03 11:30:20|2025-01-31 10:29:32|
+-------------------+-------------------+



In [32]:
churn_sdf_1.groupBy("churn_type").count().orderBy(col("count").desc()).show()

+--------------------+-----+
|          churn_type|count|
+--------------------+-----+
| CANCELAMENTO NORMAL|15462|
|   90 DIAS BLOQUEADO| 8226|
|90 DIAS PRÉ CANCE...|  460|
|90 DIAS PRÉ CANCE...|  148|
+--------------------+-----+



In [34]:
churn_sdf_1.groupBy("motivation").count().orderBy(col("count").desc()).show()

+--------------------+-----+
|          motivation|count|
+--------------------+-----+
|INADIMPLENTE-AUTOMAT| 2393|
|      NAO UTIL. SERV| 1777|
|   BLOQUEIO_REGRA_90| 1748|
|MOTIVO NAO INFORMADO| 1511|
|            ACAO GFA| 1505|
| BLOQ. AUTO. 15 DIAS| 1425|
|                NULL| 1334|
|   MIG BLOQ. EXPRESS| 1293|
| NAO ESTA UTILIZANDO| 1019|
| SOLICITACAO REVENDA|  835|
|   ACAO DSPAC/FATMIN|  742|
| REDUCAO DE DESPESAS|  706|
|          NEGATIVADO|  704|
| 018 - NAO UTIL SERV|  582|
|        BASE INADIMP|  424|
|      OUTROS MOTIVOS|  368|
|   BLOQ TRAVA MANUAL|  336|
|  ENCER DE ATIVIDADE|  291|
| ALT. POLIT DE VENDA|  289|
|       ENCER DE ATIV|  286|
+--------------------+-----+
only showing top 20 rows



 - Existem registros com churn_date anteriores a 2024, apesar do script conter uma função que filtra apenas churns a partir de 16 de abril de 2024.

1 Parte

Na primeira CTE transicao_global, a consulta seleciona os dados da tabela dev.dw_redeok_model_aux.tb_s_status_transicao (apelidada como str). Em seguida, é realizado um LEFT JOIN com a tabela dev.dw_redeok_model.tb_d_external_entity_association (apelidada como ext), estabelecendo a correspondência entre a coluna matriz de str e a coluna id_external_entity de ext. Além disso, são aplicadas as condições name_internal_entity = 'tb_d_contract' e name_external_application = 'signet', garantindo que apenas as associações relacionadas à tabela de contratos sejam consideradas.

Posteriormente, um segundo LEFT JOIN é feito com a tabela dev.dw_redeok_model.tb_d_contract (apelidada como cnt), utilizando a correspondência entre ext.uuid_internal_entity e cnt.uuid. Esse processo permite obter a UUID do contrato (uuid_contract) e incluí-la em cada linha da tabela de transição de status, associando corretamente as mudanças de status ao contrato correspondente.


2 Parte

Na segunda cte chamada bloqueio Selecionamos os campos relevantes (uuid_contract, matriz, data_transicao, previous_status, next_status, motivo_cancelamento). da cte transicao_global
Gera uma numeração (row_number()) para cada matriz, ordenando os registros pela data_transicao em ordem decrescente (desc). Isso significa que a transição mais recente receberá o número 1.
O filtro qualify mantém apenas as linhas onde row_number() = 1, ou seja, apenas a última transição registrada para cada matriz é mantida.


3 Parte

Na terceira cte chamada cancelamento_noventa_dias_bloqueio Seleciona os dados da CTE bloqueio, mantendo as colunas uuid_contract, matriz, data_transicao, previous_status, next_status e motivo_cancelamento.

Adiciona uma nova coluna data_transicao_90d, que corresponde à data_transicao acrescida de 90 dias, utilizando a função DATEADD(DAY, 90, data_transicao).

Filtra os registros onde next_status pertence ao conjunto (-7, -5, 0), ou seja, mantém apenas as transições cujo status final está dentro desses valores.

4 parte

Na quarta CTE, chamada cancelamento_normal_bloqueio_90d, não é utilizada a tabela bloqueio, mas sim todas as transições disponíveis na CTE transicao_global, sem aplicar o filtro para considerar apenas a transição mais recente.

Nessa etapa, são selecionados apenas os contratos cujo next_status seja igual a -1, indicando um cancelamento normal.

Além disso, é adicionada uma nova coluna chamada tipo_saida, com o valor 'CANCELAMENTO NORMAL', para categorizar essas transações. O objetivo dessa CTE é identificar todas as ocorrências de cancelamento normal, sem restringir a análise apenas à última transição registrada para cada matriz.transicao_global, sem filtrar apenas a mais recente.
Filtra os contratos onde next_status = -1 (cancelamento normal).
é adicionada uma coluna chamada tipo de saída onde  pois o foco é identificar qualquer transação de cancelamento normal, não apenas a mais recente



5 Parte e final

Essa parte do código faz a união dos dados tratados nas CTEs anteriores e aplica filtros finais para compor o resultado consolidado.

A primeira SELECT cria a coluna tipo_saida categorizando os contratos com next_status (-7, -5, 0) em diferentes tipos de como 90 dias bloqueado , 90 dias pré cancelado e 90 dias pré cancelado jurídico e também Apenas contratos com data_transicao_90d < current_date() são mantidos.

Em seguida ele faz o SELECT final junta os cancelamentos normais (cancelamento_normal_bloqueio_90d) com as informações adicionais do contrato (tb_d_contract) e do cliente (tb_d_customer).

Apenas contratos com churn a partir de 16/04/2024 são considerados.
Motivos específicos de cancelamento (como "TROCA DE CNPJ") são removidos.
m NOT EXISTS impede a inserção de registros duplicados já presentes na tabela tb_f_churn.
O resultado é ordenado por uuid_contract em ordem decrescente.






Oque precisaríamos para replicar o scripit da tabela churn para todo histórico?

- dev.dw_redeok_model_aux.tb_s_status_transicao : possui o status par aa denominações dos contratos
- dev.dw_redeok_model.tb_d_external_entity_association : Utilizada para fazer a associação entre matriz e uuid_contract.
- dev.dw_redeok_model.tb_d_contract : Necessária para obter uuid_contract. (porem ela so tem alguns contratos e não todos)
- dev.dw_redeok_model.tb_d_customer :Relacionada aos clientes, usada para identificar uuid_customer.



Sugestão 2

Com base na minha nova tabela, ele sugeriu extrair, para cada cliente, a última data do boleto gerado e assumir isso como churn. No entanto, ele destacou que essa é uma abordagem simplificada, já que um mesmo cliente pode ter múltiplos churns ao longo da sua jornada na ROK. Além disso, se eu me basear no conceito da tabela f_churn, os dados não vão bater.


 